## Join 2018 and 2019 iNat Data by Species Name and Create GCP UL Links

In [1]:
import json
import pandas as pd

In [2]:
# load the data
with open('train2019.json') as json_file:
    data2019 = json.load(json_file)
with open('categories2019.json') as json_file:
    cat2019 = json.load(json_file)
with open('train2018.json') as json_file:
    data2018 = json.load(json_file)
with open('categories2018.json') as json_file:
    cat2018 = json.load(json_file)

In [3]:
# check out what keys are available
data2019.keys()

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])

In [4]:
# get numeric ID from binomial name
def getID(binom_name, cat=cat2019):
    for row in cat:
        if row['name'].lower() == binom_name.lower():
            print(row['id'])

In [10]:
def getAllDirectory(year=2019, species_map=species_map):
    '''returns tuple of common name and dir'''

    data = eval("data"+str(year))
    dirs = []
    for keys in species_map.keys():
        common_name = species_map[keys][0]
        if year == 2019:
            key = species_map[keys][1][0]
        elif year == 2018:
            key = species_map[keys][1][1]
        try:
            # join and split token
            s ="/"
            # get dirname if numeric ID key matches key from species dict
            dirname = [s.join(row["file_name"].split("/")[:3]) for row in data["images"]]
            dirs.append((keys, common_name, dirname[0]))
        except:
            pass
    return dirs

dict_keys(['kingdom', 'phylum', 'name', 'family', 'genus', 'order', 'id', 'class'])

In [4]:
dirnames2019 = ["/".join(row["file_name"].split("/")[:3]) for row in data2019["images"]]
dirnames2018 = ["/".join(row["file_name"].split("/")[:3]) for row in data2018["images"]]

In [5]:
df_dir19 = pd.DataFrame(set(dirnames2019), columns=["dir19"])
df_dir19["id"]=df_dir19["dir19"].apply(lambda x: x.split("/")[2])
df_dir19["id"]=df_dir19["id"].astype("int64")

In [6]:
df_dir18 = pd.DataFrame(set(dirnames2018), columns=["dir18"])
df_dir18["id"]=df_dir18["dir18"].apply(lambda x: x.split("/")[2])
df_dir18["id"]=df_dir18["id"].astype("int64")

In [7]:
df_dir18.head()

,dir18,id
0,train_val2018/Plantae/6993,6993
1,train_val2018/Insecta/928,928
2,train_val2018/Mollusca/4673,4673
3,train_val2018/Plantae/5559,5559
4,train_val2018/Plantae/7863,7863


In [8]:
df19 = pd.read_json("categories2019.json", orient="records")

In [9]:
df18 = pd.read_json("categories2018.json", orient="records")

In [10]:
df19.head(2)

,class,family,genus,id,kingdom,name,order,phylum
0,Agaricomycetes,Amanitaceae,Amanita,0,Fungi,Amanita vaginata,Agaricales,Basidiomycota
1,Agaricomycetes,Amanitaceae,Amanita,1,Fungi,Amanita flavoconia,Agaricales,Basidiomycota


In [11]:
df_dirmerge18 = df18.merge(df_dir18,on="id", how="right")

In [12]:
df_dirmerge18.head(5)

,class,family,genus,id,kingdom,name,order,phylum,supercategory,dir18
0,Polychaeta,Amphinomidae,Hermodice,0,Animalia,Hermodice carunculata,Phyllodocida,Annelida,Animalia,train_val2018/Animalia/0
1,Polychaeta,Sabellariidae,Phragmatopoma,1,Animalia,Phragmatopoma californica,Sabellida,Annelida,Animalia,train_val2018/Animalia/1
2,Polychaeta,Sabellidae,Eudistylia,2,Animalia,Eudistylia vancouveri,Sabellida,Annelida,Animalia,train_val2018/Animalia/2
3,Polychaeta,Serpulidae,Galeolaria,3,Animalia,Galeolaria hystrix,Sabellida,Annelida,Animalia,train_val2018/Animalia/3
4,Polychaeta,Serpulidae,Serpula,4,Animalia,Serpula columbiana,Sabellida,Annelida,Animalia,train_val2018/Animalia/4


In [13]:
df_dirmerge19 = df19.merge(df_dir19,on="id", how="right")

In [179]:
dirmerge = pd.DataFrame.merge(df_dirmerge18, df_dirmerge19[["name","kingdom","phylum","class","order","family","genus","dir19"]], how="outer", on=["name"])

In [254]:
dirmerge.shape

(8517, 17)

In [180]:
# 763 more rows than dir18
dirmerge.head()

,class_x,family_x,genus_x,id,kingdom_x,name,order_x,phylum_x,supercategory,dir18,kingdom_y,phylum_y,class_y,order_y,family_y,genus_y,dir19
0,Polychaeta,Amphinomidae,Hermodice,0.0,Animalia,Hermodice carunculata,Phyllodocida,Annelida,Animalia,train_val2018/Animalia/0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Polychaeta,Sabellariidae,Phragmatopoma,1.0,Animalia,Phragmatopoma californica,Sabellida,Annelida,Animalia,train_val2018/Animalia/1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Polychaeta,Sabellidae,Eudistylia,2.0,Animalia,Eudistylia vancouveri,Sabellida,Annelida,Animalia,train_val2018/Animalia/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Polychaeta,Serpulidae,Galeolaria,3.0,Animalia,Galeolaria hystrix,Sabellida,Annelida,Animalia,train_val2018/Animalia/3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Polychaeta,Serpulidae,Serpula,4.0,Animalia,Serpula columbiana,Sabellida,Annelida,Animalia,train_val2018/Animalia/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
# translate from binomial name to common name using csv data [https://observation.org/download.php]
species_translate = pd.read_csv("species.csv",sep=";")

C:\Users\gjack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (11,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [182]:
names = species_translate[["name","Scientific name"]]

In [183]:
names.columns = ["commonname","Scientific name"]

In [184]:
_df = pd.merge(dirmerge,names, how="left", left_on="name", right_on="Scientific name")

In [185]:
df = _df.drop_duplicates(subset="name")

In [186]:
# count species without common names available
df["commonname"].isnull().sum()

2304

In [187]:
df.shape

(8517, 19)

In [188]:
# had to do this instead of normal join due to inconsistencies btwn tables
df["kingdom_x"].fillna(df["kingdom_y"], inplace=True)
df["phylum_x"].fillna(df["phylum_y"], inplace=True)
df["class_x"].fillna(df["class_y"], inplace=True)
df["order_x"].fillna(df["order_y"],inplace=True)
df["family_x"].fillna(df["family_y"], inplace=True)
df["genus_x"].fillna(df["genus_y"],inplace=True)

### Join With Yosemite Species Data

Contains data that can be used to populate gameplay tables (e.g. occurrence, abundance, nativeness)

In [64]:
# yosemite species list from NPS
yose = pd.read_csv('NPSpecies_FullList_YOSE_20200229_212528.csv')

In [66]:
yose.head()

,Park Code,Category,Order,Family,Taxa Code,Scientific Name,Synonyms,Common Name,Occurrence,Occurrence Tags,Abundance,Nativeness,Nativeness Tags,Record Status
0,YOSE,Mammal,Artiodactyla,Bovidae,891494,Ovis canadensis sierrae,NaN,Sierra Nevada bighorn sheep,Present,NaN,Rare,Native,NaN,Approved
1,YOSE,Mammal,Artiodactyla,Cervidae,731246,Cervus nannodes,NaN,NaN,Not In Park,Historical,NaN,Non-native,NaN,In Review
2,YOSE,Mammal,Artiodactyla,Cervidae,95045,Odocoileus hemionus,"Odocoileus hemionus californicus (731223),Dama...","mule deer, Mule Deer",Present,NaN,Common,Native,NaN,Approved
3,YOSE,Mammal,Artiodactyla,Suidae,95073,Sus scrofa,NaN,"pig, pig (feral), wild boar, Wild Boar",Unconfirmed,Adjacent,NaN,Non-native,NaN,Approved
4,YOSE,Mammal,Carnivora,Canidae,93340,Canis latrans,Canis latrans lestes (154884),Coyote,Present,NaN,Uncommon,Native,NaN,Approved


In [67]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [68]:
yose_cats=list(yose["Scientific Name"])

In [71]:
# example fuzzy string match
str2Match='Ovis canadensis'
Ratios = process.extractOne(str2Match, yose_cats, scorer=fuzz.partial_ratio)
print(Ratios)

('Ovis canadensis sierrae', 100)


In [189]:
df['fuzzywuzzy'] = df.name.apply(lambda x: process.extractOne(x, yose_cats, scorer=fuzz.partial_ratio))

C:\Users\gjack\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [190]:
df['yose_score']= df['fuzzywuzzy'].apply(lambda x: x[1])
df['yose_name']= df['fuzzywuzzy'].apply(lambda x: x[0])

C:\Users\gjack\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\gjack\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [193]:
df['yose_join']=None
df.loc[df.yose_score==100,'yose_join'] = df.yose_name

In [194]:
df.loc[df['yose_score']==100]['yose_join'].head()

2807    Oncorhynchus clarkii henshawi
2811              Oncorhynchus mykiss
2813                     Salmo trutta
2815            Salvelinus fontinalis
2897       Anaxyrus boreas halophilus
Name: yose_join, dtype: object

In [195]:
df1 = df.drop(columns=['kingdom_y', 'phylum_y','class_y','family_y','genus_y',
                       'Scientific name', 'fuzzywuzzy','yose_name'])

In [196]:
df1.head()

,class_x,family_x,genus_x,id,kingdom_x,name,order_x,phylum_x,supercategory,dir18,order_y,dir19,commonname,yose_score,yose_join
0,Polychaeta,Amphinomidae,Hermodice,0.0,Animalia,Hermodice carunculata,Phyllodocida,Annelida,Animalia,train_val2018/Animalia/0,NaN,NaN,Bearded fireworm,67,None
1,Polychaeta,Sabellariidae,Phragmatopoma,1.0,Animalia,Phragmatopoma californica,Sabellida,Annelida,Animalia,train_val2018/Animalia/1,NaN,NaN,NaN,88,None
2,Polychaeta,Sabellidae,Eudistylia,2.0,Animalia,Eudistylia vancouveri,Sabellida,Annelida,Animalia,train_val2018/Animalia/2,NaN,NaN,NaN,59,None
3,Polychaeta,Serpulidae,Galeolaria,3.0,Animalia,Galeolaria hystrix,Sabellida,Annelida,Animalia,train_val2018/Animalia/3,NaN,NaN,NaN,65,None
4,Polychaeta,Serpulidae,Serpula,4.0,Animalia,Serpula columbiana,Sabellida,Annelida,Animalia,train_val2018/Animalia/4,NaN,NaN,NaN,72,None


In [197]:
yose.columns

Index(['Park Code', 'Category', 'Order', 'Family', 'Taxa Code',
       'Scientific Name', 'Synonyms', 'Common Name', 'Occurrence',
       'Occurrence Tags', 'Abundance', 'Nativeness', 'Nativeness Tags',
       'Record Status'],
      dtype='object')

In [198]:
yose_sub = yose[['Scientific Name', 'Synonyms', 'Common Name', 'Occurrence',
       'Occurrence Tags', 'Abundance', 'Nativeness', 'Nativeness Tags']]

In [199]:
dfm = pd.merge(df1, yose_sub, how="left", left_on="yose_join", right_on="Scientific Name")

In [175]:
# get class counts (remove marine classes below)
# dfm["class_x"].value_counts()   # exported marine_classes to txt


### Flag Irrelevant Classes 

E.g. flag marine classes. Consider more granular separation (e.g. could remove orders like dolphins and whales from mammals)

In [200]:
marine_classes = ["Actinopterygii","Malacostraca","Bivalvia",
"Elasmobranchii","Phaeophyceae","Anthozoa","Asteroidea",
"Polyplacophora","Florideophyceae","Hexanauplia",
"Echinoidea","Cephalopoda","Polychaeta","Ascidiacea",
"Ulvophyceae","Hydrozoa","Holothuroidea","Ophiuroidea",
"Scyphozoa","Zygomycetes","Jungermanniopsida","Merostomata",
"Rhabditophora","Phascolosomatidea",
"Compsopogonophyceae","Cyanophyceae","Gymnolaemata"]

In [204]:
# mark classes we don't expect to encounter
dfm.loc[dfm['class_x'].isin(marine_classes), 'marine'] = True

In [214]:
dfm.tail()

,class_x,family_x,genus_x,id,kingdom_x,name,order_x,phylum_x,supercategory,dir18,...,yose_join,Scientific Name,Synonyms,Common Name,Occurrence,Occurrence Tags,Abundance,Nativeness,Nativeness Tags,marine
8512,Pinopsida,Pinaceae,Pinus,NaN,Plantae,Pinus echinata,Pinales,Tracheophyta,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8513,Pinopsida,Pinaceae,Pinus,NaN,Plantae,Pinus sylvestris,Pinales,Tracheophyta,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8514,Polypodiopsida,Aspleniaceae,Asplenium,NaN,Plantae,Asplenium flabellifolium,Polypodiales,Tracheophyta,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8515,Polypodiopsida,Aspleniaceae,Asplenium,NaN,Plantae,Asplenium ruta-muraria,Polypodiales,Tracheophyta,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8516,Polypodiopsida,Aspleniaceae,Asplenium,NaN,Plantae,Asplenium nidus,Polypodiales,Tracheophyta,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Create Labels

Initially set up labels to use common name where it exists, but that created issues due to length and special characters. Using binomial name instead. Will use join later to get labels that are displayed in app.

In [217]:
# set label to common name or sci name if doesn't exist
dfm['label']=dfm.name
dfm.loc[dfm["commonname"].notnull(),'label'] = dfm["commonname"]

In [219]:
dfm["label"].head()

0             Bearded fireworm
1    Phragmatopoma californica
2        Eudistylia vancouveri
3           Galeolaria hystrix
4           Serpula columbiana
Name: label, dtype: object

In [230]:
# 29 species had Common Name from yose dataset that wasn't in orig, changing label for those
dfm.loc[(dfm["commonname"].isnull())&(dfm["Common Name"].notnull()), 'label'] = dfm["Common Name"]

In [231]:
# exporting to csv
dfm.to_csv("speciesDB_full.csv",index=False)

In [232]:
# df nonmarine (exclude marine species)
df_nm = dfm[dfm.marine.isnull()]

In [301]:
# Yosemite species only table
df_yose = df_nm[df_nm.yose_join.notnull()]

In [234]:
# 680 species in iNat
df_yose.shape

(680, 24)

In [235]:
df_nm.shape

(7870, 25)

In [244]:
# df_cf stands for the cloudfile paths
df_cf = pd.read_csv(r"C:\Users\gjack\Documents\GitHub\Nature-Scavenger-Hunt\Data\iNat\cloudfiles.txt",
                    header=None, names=['pth'])

In [332]:
# total no of pics
df_cf.loc[df_cf['pth'].str.endswith("jpg")].shape[0]

704963

In [256]:
# need to remove anything that doesn't end in jpg
# check to make sure no of species makes sense (removing one extra line per species)
df_cf.shape[0]-df_cf.loc[df_cf['pth'].str.endswith("jpg")].shape[0]

8515

In [268]:
df_cfj=df_cf.loc[df_cf['pth'].str.endswith("jpg")]

In [333]:
# create column of binomial name (should match iNat name)
# df_cfj will join binomial name extracted from pth w main table
df_cfj["_biname"]=df_cfj.pth.apply(lambda x: x.split("/")[-2])
df_cfj._biname = df_cfj._biname.apply(lambda x: x.replace("_"," "))

C:\Users\gjack\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [335]:
df_cfj.head(1)

,pth,_biname
1,gs://poetic-orb-251401-vcm/Abaeis_nicippe/0228...,Abaeis nicippe


In [ ]:
# create a negative label for species not in yosemite
df_nm['neglbl'] = df_nm["name"]
df_nm.loc[df_nm["yose_join"].isnull(),"neglbl"] = "negclass"

### Export path data 
Export format is csv [filepath, label]. 

3 different setups used: 
- all species each with binomial label
- all species with non-yosemite annotated as negclass
- yosemite species only

In [372]:
# unintentionally removes one species (22 pics of salix fragilis due to ascii issue)
df_pthlbl = pd.merge(df_cfj,df_nm[["name","label","neglbl"]], left_on="_biname",right_on="name", how="inner")

In [373]:
df_pthlbl_all = df_pthlbl[['pth','name']]
df_pthlbl_neg = df_pthlbl[['pth','neglbl']]

In [374]:
df_pthlbl_neg.head()

,pth,neglbl
0,gs://poetic-orb-251401-vcm/Abaeis_nicippe/0228...,negclass
1,gs://poetic-orb-251401-vcm/Abaeis_nicippe/0278...,negclass
2,gs://poetic-orb-251401-vcm/Abaeis_nicippe/06b8...,negclass
3,gs://poetic-orb-251401-vcm/Abaeis_nicippe/0af5...,negclass
4,gs://poetic-orb-251401-vcm/Abaeis_nicippe/0cde...,negclass


In [381]:
df_pthlbl_neg.neglbl.apply(lambda x: len(x)>32).sum()

0

In [378]:
df_pthlbl_all.loc[df_pthlbl_all.name.apply(lambda x: len(x)>32),"name"] = df_pthlbl_xp.name.apply(lambda x: x[:32])

C:\Users\gjack\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\gjack\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [380]:
# check to make sure all pics have alphanumeric labels
alphachk = pd.DataFrame(df_pthlbl_all.name.str.replace("-",""))
alphachk['name'].str.replace(" ","").str.isalpha().sum()

686773

In [379]:
# all label length < 32
df_pthlbl_all.name.apply(lambda x: len(x)>32).sum()

0

In [367]:
df_pthlbl_all.to_csv("cloud_pthlblxp.csv",encoding="utf-8",header=None, index=False)

In [302]:
df_pthlbl_yoseonly = pd.merge(df_cfj,df_yose[["name","label"]], left_on="_biname",right_on="name", how="inner")

In [304]:
# ~165K images for species in Yosemite
df_pthlbl_yoseonly.shape

(165322, 4)

In [305]:
df_pthlbl_yoseonly = df_pthlbl_yoseonly[['pth','label']]

In [306]:
df_pthlbl_yoseonly.to_csv("cloud_pthlbl_yoseonly.csv",encoding="utf-8",header=None, index=False)

In [382]:
df_pthlbl_neg.to_csv("cloud_pthlbl_negcls.csv", encoding="utf-8", header=None, index=False)